In [1]:
import time
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from sklearn_extra.cluster import KMedoids

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.mixture import GaussianMixture

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import MeanShift
from sklearn.cluster import AgglomerativeClustering
#from konlpy.tag import Komoran

#import pandas_datareader as pdr
from datetime import datetime

from scipy.cluster.hierarchy import linkage 
from scipy.cluster.hierarchy import dendrogram 

from tslearn.clustering import TimeSeriesKMeans
from tslearn.metrics import dtw

import scipy.spatial.distance as ssd

#from gensim.models import Word2Vec as w_to_v

In [2]:
df = pd.read_csv('../data/df.csv', encoding = 'euc-kr')
df_price = pd.read_pickle('../fkl/price_normal.fkl')
df_dtw_matrix = pd.read_pickle('../fkl/dtw_matrix.fkl')

df_finance = pd.read_pickle('../fkl/financial_i_log_categoric.fkl')
df_price = df_price.dropna(axis = 0)

# 기업개요

In [3]:
#TFidf 변환 및 KMeans 학습

countvectorizer = CountVectorizer(stop_words=['1월', '1일','2월','3월','4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월', 'M', 'co', 'company', 'global', 'inc', '가능', '가능', '가치', '각종', 
                                              '감소', '강화', '개', '개선', '개시', '개요', '거점', '결과', '결정', '경영', 
                                              '경우', '경쟁력', '계속', '계획', '공시', '과정', '관련', '관리', '구분', 
                                              '구성', '구조', '구축', '국가', '국내', '국내외', '규모', '극대', '글로벌', 
                                              '기간', '기능', '기대', '기록', '기반', '기업', '기존', '기준', '기초',
                                              '기타', '기회', '내용', '노력', '누적', '능력', '다각', '다수', '다양', '다음',
                                              '단계', '단위', '단일', '달성', '담당', '당기', '당사', '대비', '대상', '대응',
                                              '도약', '도입', '동기', '동력', '동사', '동시', '드', '등', '마련', '매출', 
                                              '매출액', '목적', '목표', '및', '바탕', '반기', '반영', '방식', '법인', '변경',
                                              '변동', '변화', '별도', '보수', '보호', '부가', '부문', '부분', '분기', '분류',
                                              '분야', '비용', '비중', '사업', '사업장', '사용', '사항', '상기', '상반기', 
                                              '상승', '상장', '상호', '상황', '선도', '설계', '설립', '설비', '성격', '성장',
                                              '세계', '소유', '수요', '수익', '수준', '수출', '수행', '시작', '시장', '시행',
                                              '실적', '아래', '안정', '업무', '업체', '역량', '역할', '연간', '연결', '영역', 
                                              '영업', '영위', '영향', '예상', '예정', '완료', '요구', '요약', '요인', '우수', 
                                              '우위', '운영', '위주', '위치', '유가증권', '유지', '의', '이상', '이익', '이하', 
                                              '이후', '일반', '일부', '임대', '입지', '자금', '자본', '자산', '자체', '작성', 
                                              '장기', '장벽', '장점', '재무', '적극', '적용', '전개', '전년', '전년 동기', 
                                              '전략', '전망', '전문', '전반', '전체', '점유', '제고', '제외', '제품', '조정', '조직', 
                                              '종속', '종합', '주', '주도', '주력', '주식', '주식회사', '주요', '준비', '중심',
                                              '중요', '증가', '증대', '지배', '지분', '지속', '지역', '지원', '지위', '지주',
                                              '지주회사', '진행', '집중', '차별', '차지', '참고', '참여', '참조', '창출',
                                              '채널', '처리', '체계', '체제', '최고', '최근', '최대', '최소', '최초',
                                              '추가', '추세', '추진', '충족', '투자', '특성', '특수', '특징', '평가',
                                              '포함', '필요', '항목', '해당', '해외', '핵심', '향상', '향후', '현재', '현지',
                                              '현황', '형태', '확대', '확보', '확장', '활동', '활용', '회사', '효율'], 
                                  ngram_range=(1,2), max_df = 0.4, min_df = 0.05, lowercase=True, max_features = 550) #0.4 0.05
tfidftransformer = TfidfTransformer()

data = df["morph_sumdart"].copy()
data_trans = tfidftransformer.fit_transform(countvectorizer.fit_transform(data))

In [88]:
# 실제 학습
clst = KMeans(n_clusters=10, random_state=0)
clst.fit(data_trans)


clusters_text = clst.labels_.tolist()

centers_text = clst.cluster_centers_ # cluster의 중심

In [6]:
df["cluster"]= clusters_text

df_groupby_cluster = df.groupby('cluster')
#result = dict(list((df_groupby_cluster['name'])))

groupby_list = df.groupby('cluster')['labels_kse'].apply(list)

for i in groupby_list:
    print(i)
    print('\n')

['서비스업', '서비스업', '서비스업', '통신업', '서비스업', '서비스업', '전기,전자', '서비스업', '통신업', '통신업', '서비스업', '전기,전자', '서비스업', '서비스업', '서비스업', '화학', '기계']


['금융업', '화학', '철강및금속', '화학', '서비스업', '철강및금속', '화학', '화학', '유통업', '금융업', '금융업', '유통업', '금융업', '유통업', '운수장비', '유통업', '유통업', '운수장비', '화학', '금융업', '기계', '전기,전자', '전기,전자', '금융업', '화학', '서비스업', '금융업', '서비스업', '화학', '유통업', '서비스업', '서비스업', '금융업', '화학', '유통업']


['유통업', '화학', '제조업', '화학', '음식료품', '음식료품', '화학', '유통업', '음식료품', '유통업', '유통업', '유통업', '음식료품', '음식료품', '화학', '섬유,의복', '화학', '음식료품', '유통업', '제조업', '섬유,의복', '금융업', '서비스업', '섬유,의복', '음식료품', '금융업', '음식료품', '음식료품']


['은행', '금융업', '금융업', '보험', '금융업', '보험', '금융업', '은행', '증권', '금융업', '보험', '증권', '금융업', '증권', '증권', '보험', '금융업', '증권', '보험']


['운수장비', '운수장비', '운수장비', '전기,전자', '전기,전자', '화학', '기계', '서비스업', '전기,전자', '화학', '화학', '운수장비', '화학', '운수장비', '전기,전자', '철강및금속', '운수장비', '금융업', '화학']


['서비스업', '운수창고', '운수창고', '운수창고', '운수창고', '운수창고', '운수창고', '유통업', '금융업', '유통업', '운수장비', '운수창고', '철강및금속', '유통업', nan]


['철강및금속', '기계'

In [ ]:
# 데이터 프레임 형태로 클러스터 결과 확인

df["cluster"]= clusters_text

df_groupby_cluster = df.groupby('cluster')
#result = dict(list((df_groupby_cluster['name'])))

groupby_list = df.groupby('cluster')['name'].apply(list)

for i in groupby_list:
    print(i)
    print('\n')

# 주가변동 : 클러스터 15개

In [137]:
# 실제 클러스터링
n_clusters = 15
tsclst = TimeSeriesKMeans(n_clusters = n_clusters, metric = 'euclidean') #'euclidean', 'dtw'
tsclst.fit(df_price)

clusters_price = tsclst.labels_.tolist()

centers_price = tsclst.cluster_centers_

In [139]:
centers_price = centers_price.reshape(n_clusters,248)
centers_price

array([[-1.35638555, -1.35253185, -1.46126167, ...,  0.64769747,
         0.66662744,  0.65082025],
       [-0.73387025, -0.70625613, -0.95253249, ..., -0.92977463,
        -0.99271529, -0.93488664],
       [-1.3169991 , -1.30098624, -1.45987475, ..., -1.05406889,
        -1.00861374, -0.99705188],
       ...,
       [-1.52691284, -1.55979446, -1.67358752, ..., -0.31937608,
        -0.23667065, -0.3364731 ],
       [-0.62313727, -0.75958061, -1.03321526, ..., -1.06174784,
        -1.00474596, -1.08604966],
       [-1.56092688, -1.43116668, -1.46926505, ..., -1.37824023,
        -1.41306344, -1.38703223]])

In [140]:
# 데이터 프레임 형태로 클러스터 결과 확인

df_price["cluster"]= clusters_price
df_price["name"]= df_price.index

df_groupby_cluster = df_price.groupby('cluster')
#result = dict(list((df_groupby_cluster['name'])))

groupby_list = df_price.groupby('cluster')["name"].apply(list)

for i in groupby_list:
    print(i)
    print('\n')
df_price = df_price.drop("name",axis=1, errors = 'ignore')
df_price = df_price.drop("cluster",axis=1, errors = 'ignore')

['삼성바이오로직스', '카카오', 'POSCO', 'SK텔레콤', '하이브', 'HMM', '두산중공업', 'KT', 'LG유플러스', '코웨이', '현대건설', '삼성엔지니어링', '현대중공업지주', 'GS', '쌍용C&E;', '두산밥캣', 'NH투자증권', '호텔신라', '팬오션', '제일기획', '한샘', '효성티앤씨', '현대미포조선', '한전기술', '효성', '현대엘리베이', '대한전선', '한전KPS', '동국제강', '두산', '금호타이어', '코스맥스', '태광산업', '오리온홀딩스', '쿠쿠홈시스', '동원F&B;', '넥센타이어']


['셀트리온', '유한양행', '두산퓨얼셀', '에스원', '녹십자', '대웅', '종근당', '부광약품']


['삼성물산', '한국전력', '넷마블', '한미사이언스', '롯데지주', '대우조선해양', '휠라홀딩스', 'LS', '농심', '지누스', '현대홈쇼핑', '대상', 'LX하우시스']


['NAVER', 'SKC', '일진머티리얼즈', '한솔케미칼', '포스코인터내셔널', '현대로템', '한화', '한화에어로스페이스', '동원시스템즈', 'LIG넥스원']


['기아', '포스코케미칼', '대한항공', 'GS건설', 'OCI', '한화시스템', '한국항공우주', '한화생명', 'BGF리테일', 'CJ', '대우건설', '영원무역', '롯데칠성', 'CJ CGV', 'LX인터내셔널', '삼양홀딩스']


['삼성전자', 'SK하이닉스', '현대모비스', '롯데케미칼', '한온시스템', '오리온', '삼성중공업', 'GS리테일', '동서', '롯데쇼핑', '대한유화', '롯데하이마트']


['한화솔루션', 'SK케미칼', '오뚜기', 'F&F;홀딩스', 'SK디스커버리']


['KB금융', '신한지주', '하나금융지주', '삼성화재', 'S-Oil', '우리금융지주', '기업은행', '강원랜드', '삼성증권', '삼성카드', '메리츠증권', 'KCC', 'BNK금융지주', '현대해상', '

In [95]:
# 실제 클러스터링
finance_data = df_finance[["PER", "PBR", "PCR", "EV/EBITDA", "Dividend"]]
clst = KMeans(n_clusters=5)
clst.fit(finance_data)

clusters_finance = clst.labels_.tolist()

centers_finance = clst.cluster_centers_

In [ ]:
############## 3번 대안 실행

In [141]:
df_text_result = df.copy()[['name']]
df_text_result['text_clusters'] = clusters_text

df_text_result

,name,text_clusters
0,삼성전자,7
1,SK하이닉스,7
2,NAVER,0
3,삼성바이오로직스,8
4,LG화학,7
...,...,...
195,LX홀딩스,4
196,롯데하이마트,1
197,LX하우시스,4
198,삼양식품,2


In [143]:
df_price_result = pd.DataFrame()
df_price_result['name'] = df_price.index
df_price_result['price_clusters'] = clusters_price
df_price_result

,name,price_clusters
0,삼성전자,5
1,SK하이닉스,5
2,NAVER,3
3,삼성바이오로직스,0
4,LG화학,14
...,...,...
189,현대두산인프라코어,8
190,롯데하이마트,5
191,LX하우시스,2
192,삼양식품,11


In [145]:
df_finance_result = pd.DataFrame()
df_finance_result['name'] = df_finance['name']
df_finance_result['finance_clusters'] = clusters_finance
df_finance_result

,name,finance_clusters
0,삼성전자,0
1,SK하이닉스,0
2,NAVER,2
3,삼성바이오로직스,2
4,LG화학,0
...,...,...
193,현대두산인프라코어,4
194,롯데하이마트,4
195,LX하우시스,4
196,삼양식품,0


In [147]:
from scipy.spatial import distance_matrix

In [148]:
distance_matrix_text = pd.DataFrame(distance_matrix(centers_text, centers_text))
distance_matrix_text

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.472188,0.549876,0.660371,0.604808,0.612707,0.613227,0.601940,0.710100,0.590347
1,0.472188,0.000000,0.421870,0.611931,0.453208,0.490067,0.477519,0.464760,0.616506,0.441820
2,0.549876,0.421870,0.000000,0.678706,0.569061,0.562239,0.596743,0.596737,0.660370,0.565430
3,0.660371,0.611931,0.678706,0.000000,0.726047,0.728917,0.729461,0.736884,0.804335,0.678438
4,0.604808,0.453208,0.569061,0.726047,0.000000,0.611008,0.598384,0.515562,0.726476,0.554901
5,0.612707,0.490067,0.562239,0.728917,0.611008,0.000000,0.605638,0.640762,0.743594,0.604184
6,0.613227,0.477519,0.596743,0.729461,0.598384,0.605638,0.000000,0.607596,0.736299,0.583269
7,0.601940,0.464760,0.596737,0.736884,0.515562,0.640762,0.607596,0.000000,0.705550,0.597920
8,0.710100,0.616506,0.660370,0.804335,0.726476,0.743594,0.736299,0.705550,0.000000,0.719799
9,0.590347,0.441820,0.565430,0.678438,0.554901,0.604184,0.583269,0.597920,0.719799,0.000000


In [149]:
distance_matrix_price = pd.DataFrame(distance_matrix(centers_price, centers_price))
distance_matrix_price

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,22.477470,11.643519,8.741085,8.212116,18.705924,25.525158,5.089163,20.339632,12.801548,11.239583,25.154337,10.196815,14.604368,21.972076
1,22.477470,0.000000,16.251636,21.668758,20.642283,13.209900,10.615792,23.048602,17.037922,15.718601,21.996085,13.585932,19.518572,16.781702,9.893380
2,11.643519,16.251636,0.000000,16.269010,13.200379,14.216949,19.958369,14.761972,13.512407,11.615836,18.916912,19.670231,10.293241,9.585401,14.064122
3,8.741085,21.668758,16.269010,0.000000,6.420303,17.615584,24.974404,8.874277,23.582608,10.094485,8.446562,27.151349,12.641122,16.021412,21.989142
4,8.212116,20.642283,13.200379,6.420303,0.000000,14.356940,23.808009,9.186087,20.775579,7.978178,12.891006,26.817136,7.468590,12.955175,19.227336
5,18.705924,13.209900,14.216949,17.615584,14.356940,0.000000,13.643396,19.396206,14.675059,11.170640,21.607214,20.590282,11.333465,9.986565,8.683524
6,25.525158,10.615792,19.958369,24.974404,23.808009,13.643396,0.000000,25.642831,15.802802,20.274663,25.395540,11.630186,20.934713,17.547952,10.430786
7,5.089163,23.048602,14.761972,8.874277,9.186087,19.396206,25.642831,0.000000,22.263596,14.730511,9.073479,25.281085,11.672123,17.294601,23.223220
8,20.339632,17.037922,13.512407,23.582608,20.775579,14.675059,15.802802,22.263596,0.000000,18.927735,25.838677,16.993211,16.122810,11.098535,13.743269
9,12.801548,15.718601,11.615836,10.094485,7.978178,11.170640,20.274663,14.730511,18.927735,0.000000,15.662527,24.217764,10.484368,10.632531,14.367642


In [150]:
distance_matrix_finance = pd.DataFrame(distance_matrix(centers_finance, centers_finance))
distance_matrix_finance

,0,1,2,3,4
0,0.000000,6.795907,4.706058,10.510348,2.689992
1,6.795907,0.000000,10.540177,9.551765,5.083469
2,4.706058,10.540177,0.000000,14.385248,6.974976
3,10.510348,9.551765,14.385248,0.000000,9.060419
4,2.689992,5.083469,6.974976,9.060419,0.000000


In [151]:
# distance_matrix_text

In [152]:
# distance_matrix_price

In [153]:
# distance_matrix_finance

In [154]:
# df_text_result --> 기업개요 클러스터링 결과 200 x 200
# df_price_result --> 주가변동 클러스터링 결과 194 x 194
# df_finance_result --> 재무지표 클러스터링 결과 198 x 198

df_merge_result = pd.merge(df_text_result, df_price_result, left_on='name', right_on='name', how='left')
df_merge_result = pd.merge(df_merge_result, df_finance_result, left_on='name', right_on='name', how='left')
df_merge_result = df_merge_result.fillna(100)
df_merge_result = df_merge_result.astype({'price_clusters':int, 'finance_clusters':int})
df_merge_result

,name,text_clusters,price_clusters,finance_clusters
0,삼성전자,7,5,0
1,SK하이닉스,7,5,0
2,NAVER,0,3,2
3,삼성바이오로직스,8,0,2
4,LG화학,7,14,0
...,...,...,...,...
195,LX홀딩스,4,100,100
196,롯데하이마트,1,5,4
197,LX하우시스,4,2,4
198,삼양식품,2,11,0


In [155]:
df_merge_result[df_merge_result['price_clusters'] == 100]

,name,text_clusters,price_clusters,finance_clusters
10,카카오뱅크,3,100,2
14,크래프톤,0,100,2
22,SK바이오사이언스,8,100,2
29,SK아이이테크놀로지,4,100,2
115,DL이앤씨,6,100,100
195,LX홀딩스,4,100,100


In [156]:
df_dist_text = pd.DataFrame(index = df_merge_result.index, columns = df_merge_result.index)

# df_clusters_result --> 클러스터링 결과 left join으로 합친 것 200 x 200, null 값은 100으로 채움
# normal_dist_text --> text 클러스터링 distance matrix
# normal_dist_price --> 주가 클러스터링 distance matrix

# 거리의 합으로 새로운 200x200 distance matrix 만들어보자

## 가중치

for i in range(len(df_merge_result)):
    for j in range(len(df_merge_result)):
        distance = 0
        try: 
            text_cluster_1 = df_merge_result['text_clusters'][i] # 첫 번째 기업의 개요 클러스터
            text_cluster_2 = df_merge_result['text_clusters'][j] # 두 번째 기업의 개요 클러스터
            distance = distance_matrix_text[text_cluster_1][text_cluster_2] # 개요 클러스터 거리 * weight
        except KeyError:
            print('{}, {}의 text 거리가 계산되지 않습니다.'.format(df_merge_result['name'][i], df_merge_result['name'][j]))
            distance = 0
        
        df_dist_text[i][j] = distance
        

df_dist_text

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,0.60194,0.70555,0.0,0.60194,0.0,0.515562,0.515562,0.70555,...,0.596737,0.596737,0.515562,0.46476,0.59792,0.515562,0.46476,0.515562,0.596737,0.70555
1,0.0,0.0,0.60194,0.70555,0.0,0.60194,0.0,0.515562,0.515562,0.70555,...,0.596737,0.596737,0.515562,0.46476,0.59792,0.515562,0.46476,0.515562,0.596737,0.70555
2,0.60194,0.60194,0.0,0.7101,0.60194,0.0,0.60194,0.604808,0.604808,0.7101,...,0.549876,0.549876,0.604808,0.472188,0.590347,0.604808,0.472188,0.604808,0.549876,0.7101
3,0.70555,0.70555,0.7101,0.0,0.70555,0.7101,0.70555,0.726476,0.726476,0.0,...,0.66037,0.66037,0.726476,0.616506,0.719799,0.726476,0.616506,0.726476,0.66037,0.0
4,0.0,0.0,0.60194,0.70555,0.0,0.60194,0.0,0.515562,0.515562,0.70555,...,0.596737,0.596737,0.515562,0.46476,0.59792,0.515562,0.46476,0.515562,0.596737,0.70555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.515562,0.515562,0.604808,0.726476,0.515562,0.604808,0.515562,0.0,0.0,0.726476,...,0.569061,0.569061,0.0,0.453208,0.554901,0.0,0.453208,0.0,0.569061,0.726476
196,0.46476,0.46476,0.472188,0.616506,0.46476,0.472188,0.46476,0.453208,0.453208,0.616506,...,0.42187,0.42187,0.453208,0.0,0.44182,0.453208,0.0,0.453208,0.42187,0.616506
197,0.515562,0.515562,0.604808,0.726476,0.515562,0.604808,0.515562,0.0,0.0,0.726476,...,0.569061,0.569061,0.0,0.453208,0.554901,0.0,0.453208,0.0,0.569061,0.726476
198,0.596737,0.596737,0.549876,0.66037,0.596737,0.549876,0.596737,0.569061,0.569061,0.66037,...,0.0,0.0,0.569061,0.42187,0.56543,0.569061,0.42187,0.569061,0.0,0.66037


In [157]:
df_dist_price = pd.DataFrame(index = df_merge_result.index, columns = df_merge_result.index)

# df_clusters_result --> 클러스터링 결과 left join으로 합친 것 200 x 200, null 값은 100으로 채움
# normal_dist_text --> text 클러스터링 distance matrix
# normal_dist_price --> 주가 클러스터링 distance matrix

# 거리의 합으로 새로운 200x200 distance matrix 만들어보자

## 가중치

for i in range(len(df_merge_result)):
    for j in range(len(df_merge_result)):
        distance = 0
        try: 
            price_cluster_1 = df_merge_result['price_clusters'][i] # 첫 번째 기업의 개요 클러스터
            price_cluster_2 = df_merge_result['price_clusters'][j] # 두 번째 기업의 개요 클러스터
            distance = distance_matrix_price[price_cluster_1][price_cluster_2] # 개요 클러스터 거리 * weight
        except KeyError:
            distance = 0
        
        df_dist_price[i][j] = distance
        

df_dist_price

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,17.615584,18.705924,8.683524,18.705924,11.17064,11.17064,14.35694,13.2099,...,11.333465,18.705924,9.986565,18.705924,14.675059,0,0.0,14.216949,20.590282,20.590282
1,0.0,0.0,17.615584,18.705924,8.683524,18.705924,11.17064,11.17064,14.35694,13.2099,...,11.333465,18.705924,9.986565,18.705924,14.675059,0,0.0,14.216949,20.590282,20.590282
2,17.615584,17.615584,0.0,8.741085,21.989142,8.741085,10.094485,10.094485,6.420303,21.668758,...,12.641122,8.741085,16.021412,8.741085,23.582608,0,17.615584,16.26901,27.151349,27.151349
3,18.705924,18.705924,8.741085,0.0,21.972076,0.0,12.801548,12.801548,8.212116,22.47747,...,10.196815,0.0,14.604368,0.0,20.339632,0,18.705924,11.643519,25.154337,25.154337
4,8.683524,8.683524,21.989142,21.972076,0.0,21.972076,14.367642,14.367642,19.227336,9.89338,...,15.965051,21.972076,12.470926,21.972076,13.743269,0,8.683524,14.064122,16.799679,16.799679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196,0.0,0.0,17.615584,18.705924,8.683524,18.705924,11.17064,11.17064,14.35694,13.2099,...,11.333465,18.705924,9.986565,18.705924,14.675059,0,0.0,14.216949,20.590282,20.590282
197,14.216949,14.216949,16.26901,11.643519,14.064122,11.643519,11.615836,11.615836,13.200379,16.251636,...,10.293241,11.643519,9.585401,11.643519,13.512407,0,14.216949,0.0,19.670231,19.670231
198,20.590282,20.590282,27.151349,25.154337,16.799679,25.154337,24.217764,24.217764,26.817136,13.585932,...,24.103867,25.154337,21.158467,25.154337,16.993211,0,20.590282,19.670231,0.0,0.0


In [158]:
df_dist_price

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,17.615584,18.705924,8.683524,18.705924,11.17064,11.17064,14.35694,13.2099,...,11.333465,18.705924,9.986565,18.705924,14.675059,0,0.0,14.216949,20.590282,20.590282
1,0.0,0.0,17.615584,18.705924,8.683524,18.705924,11.17064,11.17064,14.35694,13.2099,...,11.333465,18.705924,9.986565,18.705924,14.675059,0,0.0,14.216949,20.590282,20.590282
2,17.615584,17.615584,0.0,8.741085,21.989142,8.741085,10.094485,10.094485,6.420303,21.668758,...,12.641122,8.741085,16.021412,8.741085,23.582608,0,17.615584,16.26901,27.151349,27.151349
3,18.705924,18.705924,8.741085,0.0,21.972076,0.0,12.801548,12.801548,8.212116,22.47747,...,10.196815,0.0,14.604368,0.0,20.339632,0,18.705924,11.643519,25.154337,25.154337
4,8.683524,8.683524,21.989142,21.972076,0.0,21.972076,14.367642,14.367642,19.227336,9.89338,...,15.965051,21.972076,12.470926,21.972076,13.743269,0,8.683524,14.064122,16.799679,16.799679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196,0.0,0.0,17.615584,18.705924,8.683524,18.705924,11.17064,11.17064,14.35694,13.2099,...,11.333465,18.705924,9.986565,18.705924,14.675059,0,0.0,14.216949,20.590282,20.590282
197,14.216949,14.216949,16.26901,11.643519,14.064122,11.643519,11.615836,11.615836,13.200379,16.251636,...,10.293241,11.643519,9.585401,11.643519,13.512407,0,14.216949,0.0,19.670231,19.670231
198,20.590282,20.590282,27.151349,25.154337,16.799679,25.154337,24.217764,24.217764,26.817136,13.585932,...,24.103867,25.154337,21.158467,25.154337,16.993211,0,20.590282,19.670231,0.0,0.0


In [159]:
df_dist_finance = pd.DataFrame(index = df_merge_result.index, columns = df_merge_result.index)

# df_clusters_result --> 클러스터링 결과 left join으로 합친 것 200 x 200, null 값은 100으로 채움
# normal_dist_text --> text 클러스터링 distance matrix
# normal_dist_price --> 주가 클러스터링 distance matrix

# 거리의 합으로 새로운 200x200 distance matrix 만들어보자

## 가중치

for i in range(len(df_merge_result)):
    for j in range(len(df_merge_result)):
        distance = 0
        try: 
            finance_cluster_1 = df_merge_result['finance_clusters'][i] # 첫 번째 기업의 개요 클러스터
            finance_cluster_2 = df_merge_result['finance_clusters'][j] # 두 번째 기업의 개요 클러스터
            distance = distance_matrix_finance[finance_cluster_1][finance_cluster_2] # 개요 클러스터 거리 * weight
        except KeyError:
            distance = 0
        
        df_dist_finance[i][j] = distance
        

df_dist_finance

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,0,2.689992,2.689992,0.0,0.0
1,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,0,2.689992,2.689992,0.0,0.0
2,4.706058,4.706058,0.0,0.0,4.706058,0.0,4.706058,10.540177,4.706058,0.0,...,6.974976,6.974976,6.974976,6.974976,6.974976,0,6.974976,6.974976,4.706058,4.706058
3,4.706058,4.706058,0.0,0.0,4.706058,0.0,4.706058,10.540177,4.706058,0.0,...,6.974976,6.974976,6.974976,6.974976,6.974976,0,6.974976,6.974976,4.706058,4.706058
4,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,0,2.689992,2.689992,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196,2.689992,2.689992,6.974976,6.974976,2.689992,6.974976,2.689992,5.083469,2.689992,6.974976,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2.689992,2.689992
197,2.689992,2.689992,6.974976,6.974976,2.689992,6.974976,2.689992,5.083469,2.689992,6.974976,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2.689992,2.689992
198,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,0,2.689992,2.689992,0.0,0.0


In [160]:
df_dist_finance

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,0,2.689992,2.689992,0.0,0.0
1,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,0,2.689992,2.689992,0.0,0.0
2,4.706058,4.706058,0.0,0.0,4.706058,0.0,4.706058,10.540177,4.706058,0.0,...,6.974976,6.974976,6.974976,6.974976,6.974976,0,6.974976,6.974976,4.706058,4.706058
3,4.706058,4.706058,0.0,0.0,4.706058,0.0,4.706058,10.540177,4.706058,0.0,...,6.974976,6.974976,6.974976,6.974976,6.974976,0,6.974976,6.974976,4.706058,4.706058
4,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,0,2.689992,2.689992,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196,2.689992,2.689992,6.974976,6.974976,2.689992,6.974976,2.689992,5.083469,2.689992,6.974976,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2.689992,2.689992
197,2.689992,2.689992,6.974976,6.974976,2.689992,6.974976,2.689992,5.083469,2.689992,6.974976,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2.689992,2.689992
198,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,0,2.689992,2.689992,0.0,0.0


In [161]:
text_mean = df_dist_text.sum().sum() / (200 * 199)

In [162]:
price_mean = df_dist_price.sum().sum() / (194 * 193)

In [163]:
finance_mean = df_dist_finance.sum().sum() / (198 * 197)

In [164]:
# 10, 14, 22, 29, (115, 195)
list1 = [10, 14, 22, 29, 115, 195]
list2 = [115, 195]
for i in range(200) :
    for j in range(200):
        if i in list1 or j in list1 :
            df_dist_price.iloc[i,j] = price_mean
        if i in list2 or j in list2 :
            df_dist_finance.iloc[i, j] = finance_mean

In [165]:
for i in list1:
    df_dist_price[i][i] = float(0)
df_dist_price

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,17.615584,18.705924,8.683524,18.705924,11.17064,11.17064,14.35694,13.2099,...,11.333465,18.705924,9.986565,18.705924,14.675059,13.145246,0.0,14.216949,20.590282,20.590282
1,0.0,0.0,17.615584,18.705924,8.683524,18.705924,11.17064,11.17064,14.35694,13.2099,...,11.333465,18.705924,9.986565,18.705924,14.675059,13.145246,0.0,14.216949,20.590282,20.590282
2,17.615584,17.615584,0.0,8.741085,21.989142,8.741085,10.094485,10.094485,6.420303,21.668758,...,12.641122,8.741085,16.021412,8.741085,23.582608,13.145246,17.615584,16.26901,27.151349,27.151349
3,18.705924,18.705924,8.741085,0.0,21.972076,0.0,12.801548,12.801548,8.212116,22.47747,...,10.196815,0.0,14.604368,0.0,20.339632,13.145246,18.705924,11.643519,25.154337,25.154337
4,8.683524,8.683524,21.989142,21.972076,0.0,21.972076,14.367642,14.367642,19.227336,9.89338,...,15.965051,21.972076,12.470926,21.972076,13.743269,13.145246,8.683524,14.064122,16.799679,16.799679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,13.145246,13.145246,13.145246,13.145246,13.145246,13.145246,13.145246,13.145246,13.145246,13.145246,...,13.145246,13.145246,13.145246,13.145246,13.145246,0.0,13.145246,13.145246,13.145246,13.145246
196,0.0,0.0,17.615584,18.705924,8.683524,18.705924,11.17064,11.17064,14.35694,13.2099,...,11.333465,18.705924,9.986565,18.705924,14.675059,13.145246,0.0,14.216949,20.590282,20.590282
197,14.216949,14.216949,16.26901,11.643519,14.064122,11.643519,11.615836,11.615836,13.200379,16.251636,...,10.293241,11.643519,9.585401,11.643519,13.512407,13.145246,14.216949,0.0,19.670231,19.670231
198,20.590282,20.590282,27.151349,25.154337,16.799679,25.154337,24.217764,24.217764,26.817136,13.585932,...,24.103867,25.154337,21.158467,25.154337,16.993211,13.145246,20.590282,19.670231,0.0,0.0


In [166]:
for i in list2:
    df_dist_finance[i][i] = float(0)
df_dist_finance

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,3.776949,2.689992,2.689992,0.0,0.0
1,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,3.776949,2.689992,2.689992,0.0,0.0
2,4.706058,4.706058,0.0,0.0,4.706058,0.0,4.706058,10.540177,4.706058,0.0,...,6.974976,6.974976,6.974976,6.974976,6.974976,3.776949,6.974976,6.974976,4.706058,4.706058
3,4.706058,4.706058,0.0,0.0,4.706058,0.0,4.706058,10.540177,4.706058,0.0,...,6.974976,6.974976,6.974976,6.974976,6.974976,3.776949,6.974976,6.974976,4.706058,4.706058
4,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,3.776949,2.689992,2.689992,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,3.776949,3.776949,3.776949,3.776949,3.776949,3.776949,3.776949,3.776949,3.776949,3.776949,...,3.776949,3.776949,3.776949,3.776949,3.776949,0.0,3.776949,3.776949,3.776949,3.776949
196,2.689992,2.689992,6.974976,6.974976,2.689992,6.974976,2.689992,5.083469,2.689992,6.974976,...,0.0,0.0,0.0,0.0,0.0,3.776949,0.0,0.0,2.689992,2.689992
197,2.689992,2.689992,6.974976,6.974976,2.689992,6.974976,2.689992,5.083469,2.689992,6.974976,...,0.0,0.0,0.0,0.0,0.0,3.776949,0.0,0.0,2.689992,2.689992
198,0.0,0.0,4.706058,4.706058,0.0,4.706058,0.0,6.795907,0.0,4.706058,...,2.689992,2.689992,2.689992,2.689992,2.689992,3.776949,2.689992,2.689992,0.0,0.0


In [167]:
df_dist_text.sum().sum()

21094.78633917349

In [168]:
df_dist_price.sum().sum()

523180.795280901

In [169]:
df_dist_finance.sum().sum()

150322.56245243238

In [170]:
ratio_text = 40000 / df_dist_text.sum().sum()

df_dist_text = df_dist_text * ratio_text
df_dist_text

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,1.141401,1.337866,0.0,1.141401,0.0,0.97761,0.97761,1.337866,...,1.131534,1.131534,0.97761,0.88128,1.133777,0.97761,0.88128,0.97761,1.131534,1.337866
1,0.0,0.0,1.141401,1.337866,0.0,1.141401,0.0,0.97761,0.97761,1.337866,...,1.131534,1.131534,0.97761,0.88128,1.133777,0.97761,0.88128,0.97761,1.131534,1.337866
2,1.141401,1.141401,0.0,1.346493,1.141401,0.0,1.141401,1.146839,1.146839,1.346493,...,1.042677,1.042677,1.146839,0.895365,1.119419,1.146839,0.895365,1.146839,1.042677,1.346493
3,1.337866,1.337866,1.346493,0.0,1.337866,1.346493,1.337866,1.377546,1.377546,0.0,...,1.252196,1.252196,1.377546,1.16902,1.364886,1.377546,1.16902,1.377546,1.252196,0.0
4,0.0,0.0,1.141401,1.337866,0.0,1.141401,0.0,0.97761,0.97761,1.337866,...,1.131534,1.131534,0.97761,0.88128,1.133777,0.97761,0.88128,0.97761,1.131534,1.337866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.97761,0.97761,1.146839,1.377546,0.97761,1.146839,0.97761,0.0,0.0,1.377546,...,1.079056,1.079056,0.0,0.859374,1.052205,0.0,0.859374,0.0,1.079056,1.377546
196,0.88128,0.88128,0.895365,1.16902,0.88128,0.895365,0.88128,0.859374,0.859374,1.16902,...,0.799951,0.799951,0.859374,0.0,0.83778,0.859374,0.0,0.859374,0.799951,1.16902
197,0.97761,0.97761,1.146839,1.377546,0.97761,1.146839,0.97761,0.0,0.0,1.377546,...,1.079056,1.079056,0.0,0.859374,1.052205,0.0,0.859374,0.0,1.079056,1.377546
198,1.131534,1.131534,1.042677,1.252196,1.131534,1.042677,1.131534,1.079056,1.079056,1.252196,...,0.0,0.0,1.079056,0.799951,1.07217,1.079056,0.799951,1.079056,0.0,1.252196


In [171]:
ratio_price = 40000 / df_dist_price.sum().sum()

df_dist_price = df_dist_price * ratio_price
df_dist_price

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,1.346807,1.430169,0.663902,1.430169,0.854056,0.854056,1.097666,1.009968,...,0.866505,1.430169,0.763527,1.430169,1.121988,1.005025,0.0,1.086963,1.574238,1.574238
1,0.0,0.0,1.346807,1.430169,0.663902,1.430169,0.854056,0.854056,1.097666,1.009968,...,0.866505,1.430169,0.763527,1.430169,1.121988,1.005025,0.0,1.086963,1.574238,1.574238
2,1.346807,1.346807,0.0,0.668303,1.681189,0.668303,0.771778,0.771778,0.490867,1.656694,...,0.966482,0.668303,1.224924,0.668303,1.803018,1.005025,1.346807,1.243854,2.075867,2.075867
3,1.430169,1.430169,0.668303,0.0,1.679884,0.0,0.978748,0.978748,0.627861,1.718524,...,0.779602,0.0,1.116583,0.0,1.555075,1.005025,1.430169,0.89021,1.923185,1.923185
4,0.663902,0.663902,1.681189,1.679884,0.0,1.679884,1.098484,1.098484,1.470034,0.756402,...,1.220614,1.679884,0.95347,1.679884,1.050747,1.005025,0.663902,1.075278,1.284426,1.284426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.005025,1.005025,1.005025,1.005025,1.005025,1.005025,1.005025,1.005025,1.005025,1.005025,...,1.005025,1.005025,1.005025,1.005025,1.005025,0.0,1.005025,1.005025,1.005025,1.005025
196,0.0,0.0,1.346807,1.430169,0.663902,1.430169,0.854056,0.854056,1.097666,1.009968,...,0.866505,1.430169,0.763527,1.430169,1.121988,1.005025,0.0,1.086963,1.574238,1.574238
197,1.086963,1.086963,1.243854,0.89021,1.075278,0.89021,0.888093,0.888093,1.00924,1.242525,...,0.786974,0.89021,0.732856,0.89021,1.033097,1.005025,1.086963,0.0,1.503895,1.503895
198,1.574238,1.574238,2.075867,1.923185,1.284426,1.923185,1.851579,1.851579,2.050315,1.038718,...,1.842871,1.923185,1.617679,1.923185,1.299223,1.005025,1.574238,1.503895,0.0,0.0


In [172]:
ratio_finance = 40000 / df_dist_finance.sum().sum()

df_dist_finance = df_dist_finance * ratio_finance
df_dist_finance

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,1.252256,1.252256,0.0,1.252256,0.0,1.808353,0.0,1.252256,...,0.715792,0.715792,0.715792,0.715792,0.715792,1.005025,0.715792,0.715792,0.0,0.0
1,0.0,0.0,1.252256,1.252256,0.0,1.252256,0.0,1.808353,0.0,1.252256,...,0.715792,0.715792,0.715792,0.715792,0.715792,1.005025,0.715792,0.715792,0.0,0.0
2,1.252256,1.252256,0.0,0.0,1.252256,0.0,1.252256,2.804683,1.252256,0.0,...,1.856002,1.856002,1.856002,1.856002,1.856002,1.005025,1.856002,1.856002,1.252256,1.252256
3,1.252256,1.252256,0.0,0.0,1.252256,0.0,1.252256,2.804683,1.252256,0.0,...,1.856002,1.856002,1.856002,1.856002,1.856002,1.005025,1.856002,1.856002,1.252256,1.252256
4,0.0,0.0,1.252256,1.252256,0.0,1.252256,0.0,1.808353,0.0,1.252256,...,0.715792,0.715792,0.715792,0.715792,0.715792,1.005025,0.715792,0.715792,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.005025,1.005025,1.005025,1.005025,1.005025,1.005025,1.005025,1.005025,1.005025,1.005025,...,1.005025,1.005025,1.005025,1.005025,1.005025,0.0,1.005025,1.005025,1.005025,1.005025
196,0.715792,0.715792,1.856002,1.856002,0.715792,1.856002,0.715792,1.352683,0.715792,1.856002,...,0.0,0.0,0.0,0.0,0.0,1.005025,0.0,0.0,0.715792,0.715792
197,0.715792,0.715792,1.856002,1.856002,0.715792,1.856002,0.715792,1.352683,0.715792,1.856002,...,0.0,0.0,0.0,0.0,0.0,1.005025,0.0,0.0,0.715792,0.715792
198,0.0,0.0,1.252256,1.252256,0.0,1.252256,0.0,1.808353,0.0,1.252256,...,0.715792,0.715792,0.715792,0.715792,0.715792,1.005025,0.715792,0.715792,0.0,0.0


In [181]:
weight_text = 4
weight_price = 5
weight_finance = 3
df_dist_new = df_dist_text*weight_text + df_dist_finance*weight_finance + df_dist_price*weight_price
df_dist_new

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,15.056406,16.259078,3.319512,15.473218,4.270279,13.605779,9.398769,14.158075,...,11.006036,13.824357,9.875451,12.823341,12.292423,11.950642,5.672496,11.492629,12.397328,13.222657
1,0.0,0.0,15.056406,16.259078,3.319512,15.473218,4.270279,13.605779,9.398769,14.158075,...,11.006036,13.824357,9.875451,12.823341,12.292423,11.950642,5.672496,11.492629,12.397328,13.222657
2,15.056406,15.056406,0.0,8.727488,16.728317,3.341516,12.181263,16.860293,10.798458,13.669441,...,14.571127,13.080232,16.27998,12.490983,19.06077,12.627556,15.8835,16.374631,18.306815,19.522078
3,16.259078,16.259078,8.727488,0.0,17.507653,5.385973,14.001971,18.81797,12.406256,8.592621,...,14.474801,10.576793,16.661105,10.244089,18.802923,13.550385,17.394934,15.529241,18.381479,13.372693
4,3.319512,3.319512,16.728317,17.507653,0.0,16.721793,5.49242,14.82792,11.26061,12.890245,...,12.776585,15.072932,10.825165,14.071916,11.936221,11.950642,8.992008,11.434207,10.948268,11.773596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,11.950642,11.950642,12.627556,13.550385,11.950642,12.627556,11.950642,8.040201,8.040201,13.550385,...,12.356425,12.356425,8.040201,11.477696,12.249021,0.0,11.477696,8.040201,12.356425,13.550385
196,5.672496,5.672496,15.8835,17.394934,8.992008,16.300312,9.942775,11.765823,11.073199,15.29393,...,7.532326,10.350648,7.255129,7.150845,8.961058,11.477696,0.0,8.872308,13.21837,14.694649
197,11.492629,11.492629,16.374631,15.529241,11.434207,14.606412,10.498283,8.498516,7.193577,17.290818,...,8.251093,8.767273,3.664279,7.888545,9.374302,8.040201,8.872308,0.0,13.983076,15.177037
198,12.397328,12.397328,18.306815,18.381479,10.948268,17.543403,13.784032,18.999178,14.567798,13.959144,...,11.36173,11.763301,14.551995,14.963104,12.932168,12.356425,13.21837,13.983076,0.0,5.008786


In [196]:
from sklearn_extra.cluster import KMedoids
kmedoids = KMedoids(n_clusters = 10, metric = 'precomputed', method = 'pam', init = 'k-medoids++', max_iter = 1000)

In [197]:
kmedoids.fit(df_dist_new)

KMedoids(init='k-medoids++', max_iter=1000, method='pam', metric='precomputed',
         n_clusters=10)

In [198]:
kmedoids_clusters = kmedoids.labels_
kmedoids_clusters

array([2, 2, 0, 0, 2, 0, 5, 5, 5, 8, 0, 3, 2, 6, 0, 9, 7, 3, 7, 6, 5, 5,
       8, 7, 4, 4, 6, 2, 0, 5, 2, 1, 0, 5, 6, 9, 0, 2, 7, 7, 3, 1, 6, 6,
       3, 2, 2, 5, 9, 4, 5, 3, 1, 4, 9, 5, 7, 4, 3, 2, 7, 4, 0, 7, 3, 6,
       0, 6, 5, 2, 8, 6, 7, 3, 6, 3, 0, 5, 0, 3, 9, 2, 3, 1, 3, 3, 2, 1,
       6, 2, 8, 1, 1, 5, 8, 8, 1, 3, 7, 8, 3, 1, 1, 6, 5, 2, 0, 1, 6, 3,
       3, 1, 1, 3, 0, 0, 3, 6, 7, 3, 0, 1, 7, 3, 5, 4, 7, 5, 3, 8, 5, 3,
       1, 3, 0, 0, 4, 8, 0, 5, 7, 7, 1, 5, 7, 2, 2, 5, 1, 3, 3, 7, 1, 9,
       2, 4, 2, 8, 2, 3, 4, 7, 7, 7, 3, 5, 9, 7, 5, 1, 8, 4, 7, 7, 5, 3,
       9, 5, 7, 7, 7, 8, 4, 4, 2, 4, 8, 4, 1, 7, 7, 3, 5, 3, 4, 5, 2, 5,
       7, 8], dtype=int64)

In [199]:
kmedoids_med = kmedoids.medoid_indices_
kmedoids_med

array([120,  41,  86,  73,  53,  68,  13, 118,   9,  35])

In [200]:
df_merge_result["merge_clusters"]= kmedoids_clusters

df_groupby_cluster = df_merge_result.groupby('merge_clusters')
#result = dict(list((df_groupby_cluster['name'])))

groupby_list = df_merge_result.groupby('merge_clusters')['name'].apply(list)

for i in groupby_list:
    print(i)
    print('\n')

['NAVER', '삼성바이오로직스', '카카오', '카카오뱅크', '크래프톤', '하이브', '두산중공업', '포스코케미칼', '삼성엔지니어링', '일진머티리얼즈', '한미사이언스', '쌍용C&E;', '씨에스윈드', '현대로템', 'DL이앤씨', '한전기술', '대한전선', '한전KPS', '후성']


['HMM', '대한항공', '현대글로비스', 'GS건설', 'CJ대한통운', '팬오션', 'OCI', '한화시스템', '한화생명', 'BGF리테일', 'CJ', '대우건설', '한샘', '대우조선해양', '영원무역', '아시아나항공', '동국제강', '롯데칠성', 'LX인터내셔널', '동원산업']


['삼성전자', 'SK하이닉스', 'LG화학', '현대모비스', '엔씨소프트', '삼성에스디에스', 'KT&G;', '한화솔루션', '롯데케미칼', '미래에셋증권', '이마트', '삼성중공업', 'GS리테일', '두산퓨얼셀', '롯데쇼핑', 'F&F;홀딩스', '한국앤컴퍼니', 'DL', '녹십자홀딩스', '대한유화', 'SK디스커버리', '롯데하이마트']


['POSCO', 'SK텔레콤', '고려아연', 'KT', 'LG유플러스', '현대건설', '현대중공업지주', 'GS', '한국가스공사', '두산밥캣', 'NH투자증권', '호텔신라', '롯데지주', 'KCC', '한국항공우주', '제일기획', '포스코인터내셔널', '효성티앤씨', '현대미포조선', '한화', '코오롱인더', '효성', 'LS', '현대엘리베이', '두산', '금호타이어', '영풍', '태광산업', '오리온홀딩스', '동원F&B;', '넥센타이어']


['한국전력', '삼성생명', '한국조선해양', 'LG디스플레이', '현대제철', '한국타이어앤테크놀로지', '한화에어로스페이스', '현대백화점', '아이에스동서', 'SK네트웍스', '화승엔터프라이즈', '풍산', '현대그린푸드', '삼양홀딩스', '현대홈쇼핑', '현대두산인프라코어']


['삼성SDI', '현대차', '기아', 'L

In [201]:
df_join = pd.merge(df, df_merge_result[['name', 'merge_clusters']], left_on='name', right_on='name', how='inner')

df_groupby_cluster = df_join.groupby('merge_clusters')
#result = dict(list((df_groupby_cluster['name'])))

groupby_list = df_join.groupby('merge_clusters')['labels_kse'].apply(list)

for i in groupby_list:
    print(i)
    print('\n')

['서비스업', '의약품', '서비스업', '은행', '서비스업', '서비스업', '기계', '비금속광물', '서비스업', '전기,전자', '금융업', '비금속광물', '기계', '운수장비', '건설업', '서비스업', '전기,전자', '건설업', '화학']


['운수창고', '운수창고', '운수창고', '건설업', '운수창고', '운수창고', '화학', '전기,전자', '보험', '유통업', '금융업', '건설업', '유통업', '운수장비', '유통업', '운수창고', '철강및금속', '음식료품', '유통업', nan]


['전기,전자', '전기,전자', '화학', '운수장비', '서비스업', '서비스업', '제조업', '화학', '화학', '증권', '유통업', '운수장비', '유통업', '전기,전자', '유통업', '섬유,의복', '금융업', '금융업', '서비스업', '화학', '금융업', '유통업']


['철강및금속', '통신업', '철강및금속', '통신업', '통신업', '건설업', '화학', '금융업', '전기가스업', '기계', '증권', '유통업', '금융업', '화학', '운수장비', '서비스업', '유통업', '화학', '운수장비', '화학', '화학', '금융업', '금융업', '기계', '서비스업', '화학', '철강및금속', '화학', '금융업', '음식료품', '화학']


['전기가스업', '보험', '운수장비', '전기,전자', '철강및금속', '화학', '운수장비', '유통업', '비금속광물', '유통업', '제조업', '철강및금속', '유통업', '금융업', '유통업', '기계']


['전기,전자', '운수장비', '운수장비', '전기,전자', '금융업', '전기,전자', '전기,전자', '기계', '화학', '서비스업', '전기,전자', '화학', '화학', '운수장비', '전기,전자', '운수장비', '화학', '전기,전자', '건설업', '의약품', '전기,전자', '철강및금속', '화학', '기계', '운수장비'

In [202]:
df_merge_result.loc[kmedoids_med]

,name,text_clusters,price_clusters,finance_clusters,merge_clusters
120,한전기술,6,0,2,0
41,대한항공,5,4,4,1
86,GS리테일,1,5,4,2
73,GS,1,0,4,3
53,LG디스플레이,9,12,4,4
68,LG이노텍,4,9,0,5
13,KB금융,3,7,1,6
118,하이트진로,2,12,0,7
9,셀트리온,8,1,2,8
35,S-Oil,1,7,3,9
